In [ ]:
!git clone https://github.com/lekshmi-j/topic-identification-nlp.git


fatal: destination path 'topic-identification-nlp' already exists and is not an empty directory.


In [ ]:
%cd topic-identification-nlp



/content/topic-identification-nlp


In [ ]:
!pip install -r requirements.txt


In [ ]:
# !pip install \
#   numpy==1.23.5 \
#   scipy==1.10.1 \
#   gensim==4.3.2 \
#   smart_open==6.4.0


In [ ]:
# import numpy as np
# import scipy.linalg as la

# # Monkey-patch for gensim compatibility
# if not hasattr(la, "triu"):
#     la.triu = np.triu


In [ ]:
# import gensim, smart_open

# print("gensim:", gensim.__version__)
# print("smart_open:", smart_open.__version__)



In [ ]:
from src.models import train_lda, train_nmf, extract_topics
from src.evaluate import compute_coherence, compute_topic_diversity


In [ ]:
import pandas as pd

df = pd.read_csv("data/processed/20newsgroups_processed.csv")
df = df.dropna(subset=["clean_text"])
texts = df["clean_text"].tolist()

print(len(texts))
df.head()


236622


,clean_text,topic
0,newsgroup_rec auto_document brown edu subject ...,rec.autos.txt
1,article boeing_com fred writes cka_uxa cso_uiu...,rec.autos.txt
2,say bought saturn would paying car saving money,rec.autos.txt
3,moreover saturn really reduce margin car even ...,rec.autos.txt
4,even people buy saturn would save money force ...,rec.autos.txt


In [ ]:
#Bag of words for LDA
from sklearn.feature_extraction.text import CountVectorizer

bow_vectorizer = CountVectorizer(
    max_df=0.5,
    min_df=20
)

X_bow = bow_vectorizer.fit_transform(texts)

print(X_bow.shape)


(236622, 7954)


In [ ]:
#TF-IDF for NMF
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(
    max_df=0.5,
    min_df=20
)

X_tfidf = tfidf_vectorizer.fit_transform(texts)

print(X_tfidf.shape)


(236622, 7954)


In [ ]:
NUM_TOPICS = 20


In [ ]:
NUM_TOPICS = 20

lda = train_lda(X_bow, NUM_TOPICS)
lda_topics = extract_topics(
    lda,
    bow_vectorizer.get_feature_names_out()
)

for i, topic in enumerate(lda_topics):
    print(f"Topic {i}: {topic}")


Topic 0: ['god', 'would', 'people', 'believe', 'one', 'say', 'christian', 'think', 'true', 'religion']
Topic 1: ['game', 'team', 'player', 'year', 'play', 'would', 'one', 'time', 'get', 'first']
Topic 2: ['file', 'program', 'use', 'line', 'get', 'window', 'code', 'copy', 'server', 'set']
Topic 3: ['armenian', 'government', 'jew', 'people', 'war', 'country', 'muslim', 'state', 'turkish', 'american']
Topic 4: ['would', 'president', 'space', 'think', 'earth', 'year', 'time', 'going', 'stephanopoulos', 'one']
Topic 5: ['com', 'writes', 'sun', 'mark', 'mike', 'dod', 'van', 'ibm', 'bill', 'netcom_com']
Topic 6: ['book', 'one', 'jesus', 'god', 'read', 'word', 'bible', 'church', 'also', 'christian']
Topic 7: ['window', 'subject', 'document', 'newsgroup_comp', 'max_max', 'graphic', 'window_misc', 'max', 'edu', 'com']
Topic 8: ['subject', 'newsgroup_rec', 'document', 'newsgroup_comp', 'hardware_document', 'edu', 'com', 'sport_hockey', 'sport_baseball', 'motorcycle_document']
Topic 9: ['car', 'on

In [1]:
#human-in-the-loop

topic_labels = {
    0: "Religion / Christianity",
    1: "Sports",
    2: "Programming / Software",
    3: "Politics / Middle East",
    4: "Politics / Space / Media",
    5: "Email / Tech Companies",
    6: "Christian Theology",
    7: "Computer Graphics / Windows",
    8: "Newsgroups / Metadata",
    9: "Automobiles / Bikes",
    10: "Images / Graphics",
    11: "Government / Products",
    12: "Marketplace / For Sale",
    13: "Israel / Human Rights",
    14: "Law / Guns",
    15: "Science Newsgroups",
    16: "Articles / Posts",
    17: "Information Requests",
    18: "Hardware / Systems",
    19: "General Discussion"
}


In [ ]:
# Document-topic distribution
doc_topic_dist = lda.transform(X_bow)

# Dominant topic per document
df["topic_id"] = doc_topic_dist.argmax(axis=1)

# Map numeric topic IDs to human-readable labels
df["topic_label"] = df["topic_id"].map(topic_labels)


In [ ]:
nmf = train_nmf(X_tfidf, NUM_TOPICS)
nmf_topics = extract_topics(
    nmf,
    tfidf_vectorizer.get_feature_names_out()
)

for i, topic in enumerate(nmf_topics):
    print(f"Topic {i}: {topic}")


/usr/local/lib/python3.12/dist-packages/sklearn/decomposition/_nmf.py:1742: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


Topic 0: ['newsgroup_talk', 'document', 'subject', 'politics_gun', 'politics_mideast', 'politics_misc', 'religion_misc', 'atf_burn', 'dividian_ranch', 'gun']
Topic 1: ['people', 'god', 'say', 'believe', 'christian', 'jesus', 'many', 'right', 'bible', 'word']
Topic 2: ['writes', 'article_apr', 'apr', 'athos_rutgers', 'virginia_edu', 'netcom_com', 'geneva_rutgers', 'gov', 'news', 'mcgill']
Topic 3: ['hardware_document', 'newsgroup_comp', 'sys_ibm', 'subject', 'sys_mac', 'problem', 'help', 'monitor', 'mac', 'question']
Topic 4: ['com', 'ibm', 'apple', 'sun', 'world_std', 'internet', 'corp', 'access_digex', 'steve_dyer', 'sandvik_newton']
Topic 5: ['newsgroup_sci', 'subject', 'electronics_document', 'med_document', 'crypt_document', 'clipper_chip', 'code', 'good', 'clipper', 'subject_tapped']
Topic 6: ['newsgroup_rec', 'subject', 'motorcycle_document', 'auto_document', 'sport_hockey', 'sport_baseball', 'document', 'bike', 'warning_please', 'read']
Topic 7: ['window', 'newsgroup_comp', 'do'

In [ ]:
# Human-in-the-loop: manual topic labels for NMF
nmf_topic_labels = {
    0: "Politics / Guns / Middle East",
    1: "Religion / Christianity",
    2: "Email / Universities / News",
    3: "Computer Hardware / Help",
    4: "Tech Companies / Internet",
    5: "Science / Cryptography / Medicine",
    6: "Recreation / Sports / Vehicles",
    7: "Software / Windows / Programs",
    8: "Religion / Social Issues",
    9: "Universities / Education",
    10: "Marketplace / For Sale",
    11: "Software / Files / Images",
    12: "Atheism / Philosophy",
    13: "Computer Graphics / Windows",
    14: "General Requests",
    15: "Help / Information Requests",
    16: "Science / Space",
    17: "General Discussion",
    18: "Articles / News Posts",
    19: "General Opinions"
}


In [ ]:
nmf_doc_topic_dist = nmf.transform(X_tfidf)


In [ ]:
df["nmf_topic_id"] = nmf_doc_topic_dist.argmax(axis=1)


In [ ]:
df["nmf_topic_label"] = df["nmf_topic_id"].map(nmf_topic_labels)


In [ ]:
lda_coh = compute_coherence(lda_topics, tokenized_texts)
nmf_coh = compute_coherence(nmf_topics, tokenized_texts)

lda_div = compute_topic_diversity(lda_topics)
nmf_div = compute_topic_diversity(nmf_topics)

print("LDA Coherence:", lda_coh)
print("NMF Coherence:", nmf_coh)
print("LDA Diversity:", lda_div)
print("NMF Diversity:", nmf_div)


In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(
    n_components=NUM_TOPICS,
    max_iter=10,
    learning_method="batch",
    random_state=42
)

lda.fit(X_bow)


KeyboardInterrupt: 

In [ ]:
'''import numpy as np

def get_top_words(model, feature_names, n_top_words=10):
    topics = []
    for topic_idx, topic in enumerate(model.components_):
        top_words = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        topics.append(top_words)
    return topics

lda_topics = get_top_words(
    lda,
    bow_vectorizer.get_feature_names_out()
)

for i, topic in enumerate(lda_topics):
    print(f"Topic {i}: {topic}")


In [ ]:
from sklearn.decomposition import NMF

nmf = NMF(
    n_components=NUM_TOPICS,
    random_state=42
)

nmf.fit(X_tfidf)


In [ ]:
nmf_topics = get_top_words(
    nmf,
    tfidf_vectorizer.get_feature_names_out()
)

for i, topic in enumerate(nmf_topics):
    print(f"Topic {i}: {topic}")


In [ ]:
lda_doc_topics = lda.transform(X_bow)
nmf_doc_topics = nmf.transform(X_tfidf)

lda_doc_topics.shape


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

topic_strength = np.sum(lda_doc_topics, axis=0)

plt.bar(range(NUM_TOPICS), topic_strength)
plt.xlabel("Topic Index")
plt.ylabel("Total Weight")
plt.title("Topic Frequency (LDA)")
plt.show()


In [ ]:
'''def top_docs_per_topic(doc_topic_dist, texts, topic_idx, n=3):
    top_doc_indices = doc_topic_dist[:, topic_idx].argsort()[-n:][::-1]
    return [texts[i][:300] for i in top_doc_indices]

for i in range(3):
    print(f"\n--- Topic {i} sample docs ---")
    for doc in top_docs_per_topic(lda_doc_topics, texts, i):
        print(doc)


Generative Models:
LDA assumes documents are generated from latent topic distributions, making it a probabilistic generative model.

Topic Distributions:
Each document is represented as a mixture of topics rather than belonging to a single topic.

Interpretability vs Accuracy:
NMF often produces more interpretable topics, while LDA provides a principled probabilistic framework.


In [ ]:
from gensim.corpora.dictionary import Dictionary
from gensim.models.coherencemodel import CoherenceModel


In [ ]:
tokenized_texts = [text.split() for text in df["clean_text"]]

dictionary = Dictionary(tokenized_texts)
corpus = [dictionary.doc2bow(text) for text in tokenized_texts]


In [ ]:
'''def sklearn_topics_to_gensim(topics):
    return [[word for word in topic] for topic in topics]


In [ ]:
lda_gensim_topics = sklearn_topics_to_gensim(lda_topics)

lda_coherence = CoherenceModel(
    topics=lda_gensim_topics,
    texts=tokenized_texts,
    dictionary=dictionary,
    coherence="c_v"
).get_coherence()

print("LDA Coherence:", lda_coherence)


In [ ]:
nmf_gensim_topics = sklearn_topics_to_gensim(nmf_topics)

nmf_coherence = CoherenceModel(
    topics=nmf_gensim_topics,
    texts=tokenized_texts,
    dictionary=dictionary,
    coherence="c_v"
).get_coherence()

print("NMF Coherence:", nmf_coherence)


In [ ]:
'''def topic_diversity(topics, top_k=10):
    unique_words = set()
    total_words = 0

    for topic in topics:
        words = topic[:top_k]
        unique_words.update(words)
        total_words += len(words)

    return len(unique_words) / total_words


In [ ]:
lda_diversity = topic_diversity(lda_topics)
nmf_diversity = topic_diversity(nmf_topics)

print("LDA Topic Diversity:", lda_diversity)
print("NMF Topic Diversity:", nmf_diversity)


Model Comparison:

LDA:
- Probabilistic and generative
- Topics are broader
- Lower coherence but theoretically grounded

NMF:
- Matrix factorization based
- Topics are more interpretable
- Higher coherence and diversity

Overall, NMF produced cleaner and more interpretable topics for this dataset.


Why Accuracy Does Not Work for Topic Models:
- There is no single correct topic per document
- Topic boundaries are subjective
- Evaluation is qualitative and exploratory

Human-in-the-loop NLP:
- Humans interpret and validate topics
- Automated metrics only assist evaluation
- Domain knowledge is essential
